In [70]:
from gurobipy import *
from pulp import *

In [71]:
# Reading instance
'''
    read_instance(file_name: str)
    read a instance file and return it values
    :return: (n, m, graph, req), where
    n: int - number of vertices
    m: int - number of edges
    graph: [[int]] - an adjacency matrix
    req: [[float]] - an matrix of requirements (sup trian)
'''
def read_instance(file_name: str) -> (int, int, [[int]], [[float]]):
    with open(file_name, 'r') as instance_file:
        (n, m) = map(lambda x: int(x), instance_file.readline().split())
        graph = [[ -1 for _ in range(n)] for _ in range(n)]
        req = [[ 0.0 for _ in range(n)] for _ in range(n)]
        for _ in range(m):
            # length(u,v) = l
            (u, v, l) = map(lambda x: int(x), instance_file.readline().split())
            graph[u][v] = l
            graph[v][u] = l
        # main diagonal equals 0
        for i in range(n):
            graph[i][i] = -1
        for i in range(n):
            for j in range(i,n):
                if j == i:
                    req[i][i] = 0.0
                else:
                    r = float(instance_file.readline())
                    req[i][j] = r
                    req[j][i] = r
        
    return (n, m, graph, req)

In [72]:
'''
    get_o_u(req)
    Generate O value for each vertice u
    return: o, an [float]
'''
def get_o_u(req) -> [[float]]:
    o = []
    for u in range(n):
        o.append(sum(req[u][v] for v in range(n)))
    return o


In [73]:
'''
    defining_vars(graph: [[int]])
    generate MIP vars
    return: (x, y, f), where:
    x: dict(LPVar): binary x[i][j]
    y: dict(LPVar): binary y[u][i][j]
    f: dict(LPVar): cont   f[u][i][j]
'''
def defining_vars(graph: [[int]], n: int) -> (dict, dict, dict):
    x = LpVariable.dicts("x", [(i,j) for i in range(n) for j in range(i, n) if graph[i][j] >= 0],
                        lowBound=0, upBound=1, cat='Integer')
    y = LpVariable.dicts("y", [(u,i,j) for u in range(n) for i in range(n) for j in range(n) if graph[i][j] >= 0],
                        lowBound=0, upBound=1, cat='Integer')
    f = LpVariable.dicts("f", [(u,i,j) for u in range(n) for i in range(n) for j in range(n) if graph[i][j] >= 0],
                        lowBound=0)
    return (x, y, f)

In [74]:
'''
    defining_objective(graph: [[int]], f: dict)
    Generate objective function
    return: problem, an LpProblem
'''
def defining_objective(graph: [[int]], n: int, f: dict):
    problem = LpProblem("OCT problem", LpMinimize)
    problem += lpSum(graph[i][j] * (f[u, i, j]) for u in range(n) for i in range(n) for j in range(n) if graph[i][j] >= 0), "communication_cost"
    return problem

In [75]:
def first_constraint(n, o, problem, f): # equivalente: 2.13
    for u in range(n):
        problem += lpSum(f[u, u, j] for j in range(n) 
                         if (j != u) and (graph[u][j] >= 0)) == o[u]
    return problem

In [106]:
def second_constraint(n, req, problem, f): # equivalente: 2.12
    for u in range(n):
        for i in range(n):
            if i==u: continue
            problem += lpSum(f[u, i, j] - f[u, j, i] for j in range(n) 
                             if (j != u) and (j != i) and (graph[i][j] >= 0)) == - req[u][i]
    return problem

In [114]:
def third_fourth_constraint(n, o, problem, f, y): # não tem equivalente
    for u in range(n):
        min_r = min(req[u][i] for i in range(n) if req[u][i] > 0)
        M = o[u] - min_r
        for i in range(n):
            for j in range(i, n):
                if i==j or (graph[i][j] < 0): continue
                problem += f[u,i,j] <= M * y[u,i,j]
                problem += f[u,j,i] <= M * y[u,j,i]
    return problem

In [112]:
def fifth_constraint(n, problem, y): # não tem equivalente
    for u in range(n):
        problem += lpSum(y[u, i, j] for i in range(n) for j in range(n) 
                         if (j != u) and (j != i) and (graph[i][j] >= 0)) == n-1
    return problem

In [79]:
def sixth_constraint(n, problem, x, y): # não tem equivalente
    for u in range(n):
        for i in range(n):
            for j in range(i, n):
                if (graph[i][j] < 0): continue
                problem += y[u,i,j] + y[u,j,i] <= x[i,j]
    return problem

In [80]:
def alternative_constraint(n, o, problem, x, f): # equivalente: 2.14
    for u in range(n):
        for i in range(n):
            for j in range(i,n):
                if (graph[i][j] < 0): continue
                problem += f[u,i,j] + f[u,j,i] <= o[u] * x[i,j]
    return problem

In [81]:
def seventh_constraint(n, problem, x): # equivalente: 2.10
    problem += lpSum(x[i,j] for i in range(n) for j in range(i,n) 
                     if (i != j) and (graph[i][j] >= 0)) == n-1
    return problem

In [82]:
def extra_constraint(n, problem, f): # equivalente: 2.11
    for u in range(n):
        problem += lpSum(f[u, j, u] for j in range(u,n) 
                         if (j != u) and (graph[u][j] >= 0)) == 0
    return problem

In [107]:
(n, m, graph, req) = read_instance('./instances/well-known/berry6.in')
o = get_o_u(req)

In [108]:
(x, y, f) = defining_vars(graph, n)

In [115]:
problem = defining_objective(graph, n, f)
problem = first_constraint(n, o, problem, f)
# problem = extra_constraint(n, problem, f)
problem = second_constraint(n, req, problem, f)
problem = third_fourth_constraint(n, o, problem, f, y) # Remover caso use alternativa
problem = fifth_constraint(n, problem, y) # Remover caso use alternativa
problem = sixth_constraint(n, problem, x, y) # Remover caso use alternativa
# problem = alternative_constraint(n, o, problem, x, f)
problem = seventh_constraint(n, problem, x)

In [116]:
solver = GUROBI(timeLimit = 3600)
solver.buildSolverModel(problem)
solver.callSolver(problem)
solver.findSolutionValues(problem)

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 313 rows, 375 columns and 1065 nonzeros
Model fingerprint: 0x2818b6e4
Variable types: 180 continuous, 195 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+01]
Presolve removed 30 rows and 60 columns
Presolve time: 0.00s
Presolved: 283 rows, 315 columns, 975 nonzeros
Variable types: 150 continuous, 165 integer (165 binary)

Root relaxation: infeasible, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (51 simplex iterations) in 0.02 seco

<ipython-input-116-13dbcef67e8e>:2: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  solver.buildSolverModel(problem)


-1

In [117]:
problem

OCT_problem:
MINIMIZE
3*f_(0,_0,_1) + 6*f_(0,_0,_2) + 5*f_(0,_0,_3) + 9*f_(0,_0,_4) + 7*f_(0,_0,_5) + 3*f_(0,_1,_0) + 3*f_(0,_1,_2) + 2*f_(0,_1,_3) + 4*f_(0,_1,_4) + 8*f_(0,_1,_5) + 6*f_(0,_2,_0) + 3*f_(0,_2,_1) + 3*f_(0,_2,_3) + 7*f_(0,_2,_4) + 2*f_(0,_2,_5) + 5*f_(0,_3,_0) + 2*f_(0,_3,_1) + 3*f_(0,_3,_2) + 9*f_(0,_3,_4) + 2*f_(0,_3,_5) + 9*f_(0,_4,_0) + 4*f_(0,_4,_1) + 7*f_(0,_4,_2) + 9*f_(0,_4,_3) + 1*f_(0,_4,_5) + 7*f_(0,_5,_0) + 8*f_(0,_5,_1) + 2*f_(0,_5,_2) + 2*f_(0,_5,_3) + 1*f_(0,_5,_4) + 3*f_(1,_0,_1) + 6*f_(1,_0,_2) + 5*f_(1,_0,_3) + 9*f_(1,_0,_4) + 7*f_(1,_0,_5) + 3*f_(1,_1,_0) + 3*f_(1,_1,_2) + 2*f_(1,_1,_3) + 4*f_(1,_1,_4) + 8*f_(1,_1,_5) + 6*f_(1,_2,_0) + 3*f_(1,_2,_1) + 3*f_(1,_2,_3) + 7*f_(1,_2,_4) + 2*f_(1,_2,_5) + 5*f_(1,_3,_0) + 2*f_(1,_3,_1) + 3*f_(1,_3,_2) + 9*f_(1,_3,_4) + 2*f_(1,_3,_5) + 9*f_(1,_4,_0) + 4*f_(1,_4,_1) + 7*f_(1,_4,_2) + 9*f_(1,_4,_3) + 1*f_(1,_4,_5) + 7*f_(1,_5,_0) + 8*f_(1,_5,_1) + 2*f_(1,_5,_2) + 2*f_(1,_5,_3) + 1*f_(1,_5,_4) + 3*f_(2,_0,_1) + 6*